# 👋🌍 Hello, world: Submit an optimization job to 1QBit

This notebook will walk through how to use Azure Quantum to submit optimization problems to [1QBit](https://1qbit.com/).

## Submit a simple job to 1QBit using Azure Quantum

Let's begin. All code in this example will be written in Python. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace
The first step is connecting to your Azure Quantum workspace. To do so, simply create a `Workspace` object as seen in the following code cell. The parameters corresponding to your workspace should have been automatically entered for you.

In [ ]:
from azure.quantum import Workspace

workspace = Workspace (
    subscription_id = "",
    resource_group = "",
    name = "",
    location = ""
)

Let's use the `Workspace` object to view the available _targets_.

In [ ]:
targets = workspace.get_targets()
print("This workspace's targets:")
for target in targets:
    print("-", target.name)


The resulting list should contain a list of target names, along with some details for each one. A _target_ is a remote solver that can perform optimization or quantum computation. In this example, we will focus on optimization.

### ❕ Do you see `1qbit.tabu` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add 1QBit to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the 1QBit provider.

### 1QBit: The optimization provider
Azure Quantum connects with first- and third-party optimizers to deliver optimization solutions. These offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for 1QBit's targets.

Optimization Algorithm | Target ID | Python Class
| --- | ---| ---| --- | -- |
[Tabu search](https://docs.microsoft.com/azure/quantum/provider-1qbit#tabu-search-solver) | `1qbit.tabu` |  `azure.quantum.optimization.oneqbit.TabuSearch` |
[Path relinking](https://docs.microsoft.com/azure/quantum/optimization-parallel-tempering) | `1qbit.pticm`  | `azure.quantum.optimization.oneqbit.PathRelinkingSolver`
[Parallel tempering with isoenergetic cluster moves](https://docs.microsoft.com/azure/quantum/provider-1qbit#pticm-solver) | `1qbit.pathrelinking`  | `azure.quantum.optimization.oneqbit.PticmSolver`

For this example, we will use `1qbit.tabu`. To learn more about 1QBit's targets, check out our [documentation](https://docs.microsoft.com/en-us/azure/quantum/provider-1qbit#pticm-solver).

### 2. Define an optimization job
We will demonstrate how to use Azure Quantum for optimization by submitting a very simple cost function. Let's define one that contains just two terms.

In [ ]:
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term

terms = [Term(c=1, indices=[1, 0]), Term(c=2, indices=[0, 1])]
problem = Problem(name="Optimization problem", problem_type=ProblemType.ising, terms=terms)

print("Name:", problem.name)
print("Number of terms:", len(problem.terms))
print("Problem type:", problem.problem_type)

You should see the problem details in the output. Note that this problem uses `ProblemType.ising`, which means it uses an [Ising model](https://docs.microsoft.com/azure/quantum/optimization-concepts-ising-model-for-optimization). [Binary optimization](https://docs.microsoft.com/azure/quantum/optimization-binary-optimization) formulations (such as PUBO) are also supported. Read more about problem types [here](https://docs.microsoft.com/azure/quantum/optimization-problem-type).  

### 3. Submit the job
Now that we've defined a problem, we can submit it to Azure Quantum. Note that this will consume a few seconds of computation time, and depending on the offer you chose when adding the 1QBit provider, charges may apply.

In [ ]:
from azure.quantum.optimization.oneqbit import TabuSearch
import time

# Instantiate a solver to solve the problem. 
solver = TabuSearch(workspace, timeout=100)

# Optimize the problem.
print('Submitting problem...')
start = time.time()
result = solver.optimize(problem)
time_elapsed = time.time() - start
solutions = result['solutions']

# Print the solution(s).
print(f'\nObtained result in {time_elapsed} seconds.')
print(f'Computed {len(solutions)} solution(s).')
for i in range(len(solutions)):
    print(f'Solution {i}:')
    print('\tCost:', solutions[i]['cost'])
    print('\tConfiguration:', solutions[i]['configuration'])

**See a solutiuon listed above? Congratulations, you've submitted an optimization job with Azure Quantum! 👏**

### 4. Next steps
Next, you can try submitting an optimization job to other solvers that 1QBit offers. Or try submitting to another provider entirely by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.